In [1]:
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
from config import gkey

csv_path = "../data/Stanford_Data.csv"
solar_df = pd.read_csv(csv_path, delimiter=";" , encoding="ANSI")
solar_df.head()

,Unnamed: 0,tile_count,solar_system_count,total_panel_area,fips,average_household_income,county,education_bachelor,education_college,education_doctoral,...,incentive_count_nonresidential,incentive_residential_state_level,incentive_nonresidential_state_level,net_metering,feedin_tariff,cooperate_tax,property_tax,sales_tax,rebate,avg_electricity_retail_rate
0,0,0,0,0.000000,27145011200,70352.78987,Stearns County,569,1690,13,...,39,11,13,34,0,0,25,12,0,9.46
1,1,25,21,1133.436461,27145011301,61727.08520,Stearns County,674,1434,108,...,39,11,13,34,0,0,25,12,0,9.46
2,2,3,3,64.505776,27145011302,71496.88658,Stearns County,854,1459,31,...,39,11,13,34,0,0,25,12,0,9.46
3,3,0,0,0.000000,27145011304,86840.15275,Stearns County,640,1116,68,...,39,11,13,34,0,0,25,12,0,9.46
4,4,5,5,164.583303,27145011400,89135.31560,Stearns County,654,1314,15,...,39,11,13,34,0,0,25,12,0,9.46


In [2]:
#sort values for each column to determine which ones carry the most NA values
solar_df.count().sort_values(ascending=True).head()

voting_2012_dem_percentage    61983
voting_2012_gop_percentage    61983
cooling_design_temperature    66735
elevation                     66735
heating_design_temperature    66735
dtype: int64

In [3]:
#delete 2012 dem and gop voting percentage as these have largest contribution to NA data
del solar_df["voting_2012_dem_percentage"]
del solar_df["voting_2012_gop_percentage"]

In [4]:
#drop all NA values from remaining data set
solar_df = solar_df.dropna(how='any')

In [5]:
#sort values on remaining dataframe to ensure all columns contain same number of rows with data
solar_df.count().sort_values(ascending=True)

Unnamed: 0                                 63847
age_more_than_85_rate                      63847
age_75_84_rate                             63847
age_35_44_rate                             63847
age_45_54_rate                             63847
age_65_74_rate                             63847
age_55_64_rate                             63847
age_10_14_rate                             63847
age_15_17_rate                             63847
age_5_9_rate                               63847
household_type_family_rate                 63847
dropout_16_19_inschool_rate                63847
occupation_construction_rate               63847
occupation_public_rate                     63847
occupation_information_rate                63847
occupation_finance_rate                    63847
occupation_education_rate                  63847
occupation_administrative_rate             63847
age_25_34_rate                             63847
occupation_manufacturing_rate              63847
age_18_24_rate      

In [6]:
#export cleaned data to csv

solar_df.to_csv("../data/Cleaned_Data.csv", index=False, header=True, encoding="ANSI")

In [7]:
#import cleaned data csv file

csv_path_clean = "../data/Cleaned_Data.csv"
solar_clean_df = pd.read_csv(csv_path_clean, delimiter="," , encoding="ANSI")
solar_clean_df.head()

,Unnamed: 0,tile_count,solar_system_count,total_panel_area,fips,average_household_income,county,education_bachelor,education_college,education_doctoral,...,incentive_count_nonresidential,incentive_residential_state_level,incentive_nonresidential_state_level,net_metering,feedin_tariff,cooperate_tax,property_tax,sales_tax,rebate,avg_electricity_retail_rate
0,0,0,0,0.000000,27145011200,70352.78987,Stearns County,569,1690,13,...,39,11,13,34,0,0,25,12,0,9.46
1,2,3,3,64.505776,27145011302,71496.88658,Stearns County,854,1459,31,...,39,11,13,34,0,0,25,12,0,9.46
2,3,0,0,0.000000,27145011304,86840.15275,Stearns County,640,1116,68,...,39,11,13,34,0,0,25,12,0,9.46
3,4,5,5,164.583303,27145011400,89135.31560,Stearns County,654,1314,15,...,39,11,13,34,0,0,25,12,0,9.46
4,5,0,0,0.000000,27145011500,62225.90361,Stearns County,522,1395,24,...,39,11,13,34,0,0,25,12,0,9.46


In [8]:
#check that size of new csv is trimmed to ensure it has been cleaned
solar_clean_df.shape

(63847, 167)

In [9]:
#get unique lat/lon combinations for each county

#group dataframe by county and state

grouped_location = solar_clean_df.groupby(["county", "state"])

#create empty dataframe for sorted county and state data

county_df=pd.DataFrame()

#create new column for total population in each county
county_df["population"]=grouped_location["population"].sum()

#create new columns for Stanford study reported lat/lon
county_df["lat"]=grouped_location["lat"].mean()
county_df["lon"]=grouped_location["lon"].mean()

#create empty columns as placeholder for google determined lat/lon for each county
county_df["lat_gmap"]=""
county_df["lon_gmap"]=""

#reset index on county data frame so county names repeat

county_df=county_df.reset_index()
county_df.head()


,county,state,population,lat,lon,lat_gmap,lon_gmap
0,Abbeville County,sc,16713,34.301500,-82.426750,,
1,Acadia Parish,la,44719,30.249556,-92.393000,,
2,Accomack County,va,33115,37.784250,-75.649750,,
3,Ada County,id,417501,43.607966,-116.273847,,
4,Adair County,ia,4561,41.280000,-94.485500,,


In [10]:
#determine lat/lon for each county and state from geocode

#create params array that contains google api key
params = {"key": gkey}

#for each row in county dataframe, identify the base url
for index, row in county_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"    
    
    #identify the county and state combination in each row
    county_lookup=row["county"]
    
    #ensure state abbreviation is upper case
    state_lookup=row["state"].upper()
    
    #create params column of combined county, state address pairs
    params['address'] = str(f"{county_lookup},{state_lookup}")

    # Execute request
    counties_lat_lng = requests.get(base_url, params=params).json()
    
     # Add resulting lat & lng to their respective columns at the current iteration's row index
    county_df.loc[index, "lat_gmap"] = counties_lat_lng["results"][0]["geometry"]["location"]["lat"]
    county_df.loc[index, "lon_gmap"] = counties_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
county_df.head()


,county,state,population,lat,lon,lat_gmap,lon_gmap
0,Abbeville County,sc,16713,34.301500,-82.426750,34.1891,-82.4753
1,Acadia Parish,la,44719,30.249556,-92.393000,30.2297,-92.3814
2,Accomack County,va,33115,37.784250,-75.649750,37.7063,-75.8069
3,Ada County,id,417501,43.607966,-116.273847,43.4788,-116.242
4,Adair County,ia,4561,41.280000,-94.485500,41.2742,-94.48


In [13]:

#export county level coordination to csv file so as to not run google API pull every time code is executed in the future
county_df.to_csv("../data/County_Coordinates.csv", index=False, header=True, encoding="ANSI")